In [5]:
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Activation, Dropout, Flatten, Dense



def train_conv(target):
    # Train on the raw data (not dim reduced).
    data = np.load("train.npz", allow_pickle=True)

    # The number of output neurons at the last layer is equal to the number of categories we're classifying into.
    if target == "resp":
        noutput = 2
    elif target == "conf":
        noutput = 3
    elif target == "condition":
        noutput = 5

    nsub = np.max(data['sub']) + 1
    min_trials_per_class = 30

    scores = np.full((nsub), np.nan)
    for sno in range(nsub):
        # Find this participant's data.
        input_sub = data['input'][data['sub'] == sno, :, :]
        input_sub[np.isnan(input_sub)] = 0
        target_sub = data[target][data['sub'] == sno]

        # Convert string labels to int.
        if target == "condition":
            _, target_sub = np.unique(target_sub, return_inverse=True)

        # Split trials randomly into training and test/validation data.
        input_sub_train, input_sub_valid, target_sub_train, target_sub_valid = train_test_split(input_sub, target_sub, test_size=0.2)
        # Compute weights for each target class. This is especially important for confidence data, which are highly unbalanced.
        # For example if we had 2 classes and there were 50 trials of class 0 and 100 trials of class 2, then the weights would
        # be (0.666, 0.333), i.e. the class with half as many trials would be weighted twice as much when fitting the network.
        utarget_sub = np.unique(target_sub_train)
        weights = class_weight.compute_class_weight("balanced", classes=utarget_sub, y=target_sub_train)
        weights = dict(zip(utarget_sub, weights/np.sum(weights)))

        # Build the network.
        # [YOUR CODE HERE]
        model = Sequential()
        model.add(Conv1D(32,3,input_shape=(210,62)))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size = 2))
        
        model.add(Conv1D(32,3))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size = 2))
         
        model.add(Conv1D(64,3))
        model.add(Activation('relu'))
        model.add(MaxPooling1D(pool_size = 2))      
        
        model.add(Flatten())
        model.add(Dense(64))
        model.add(Activation('relu'))
        model.add(Dropout(0.5))
        model.add(Dense(noutput))
        model.add(Activation('softmax'))
        
        ## model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
        # [After this short block of code, there should be a variable called "model"]

        # Fit the network and store weighted accuracy for the validation data from the last epoch. 
        model.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.SparseCategoricalCrossentropy(), weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()])
        
        # model summary
        print(model.summary())
        history = model.fit(x=input_sub_train, y=target_sub_train, verbose=2, class_weight=weights, validation_data=(input_sub_valid, target_sub_valid), batch_size=32, epochs=100)
        scores[sno] = history.history['val_sparse_categorical_accuracy'][-1]

    print('Balanced accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

print("hello world")
train_conv("conf")



hello world
Model: "sequential_43"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_127 (Conv1D)          (None, 208, 32)           5984      
_________________________________________________________________
activation_211 (Activation)  (None, 208, 32)           0         
_________________________________________________________________
max_pooling1d_127 (MaxPoolin (None, 104, 32)           0         
_________________________________________________________________
conv1d_128 (Conv1D)          (None, 102, 32)           3104      
_________________________________________________________________
activation_212 (Activation)  (None, 102, 32)           0         
_________________________________________________________________
max_pooling1d_128 (MaxPoolin (None, 51, 32)            0         
_________________________________________________________________
conv1d_129 (Conv1D)          (None, 49, 6

Epoch 45/100
 - 1s - loss: 0.0911 - sparse_categorical_accuracy: 0.6594 - val_loss: 2.2535 - val_sparse_categorical_accuracy: 0.6609
Epoch 46/100
 - 1s - loss: 0.1147 - sparse_categorical_accuracy: 0.6622 - val_loss: 1.5759 - val_sparse_categorical_accuracy: 0.6635
Epoch 47/100
 - 1s - loss: 0.1127 - sparse_categorical_accuracy: 0.6648 - val_loss: 2.1097 - val_sparse_categorical_accuracy: 0.6659
Epoch 48/100
 - 1s - loss: 0.0979 - sparse_categorical_accuracy: 0.6673 - val_loss: 1.9965 - val_sparse_categorical_accuracy: 0.6686
Epoch 49/100
 - 1s - loss: 0.0853 - sparse_categorical_accuracy: 0.6701 - val_loss: 2.2406 - val_sparse_categorical_accuracy: 0.6716
Epoch 50/100
 - 1s - loss: 0.0905 - sparse_categorical_accuracy: 0.6732 - val_loss: 2.0191 - val_sparse_categorical_accuracy: 0.6744
Epoch 51/100
 - 1s - loss: 0.0977 - sparse_categorical_accuracy: 0.6759 - val_loss: 2.4372 - val_sparse_categorical_accuracy: 0.6772
Epoch 52/100
 - 1s - loss: 0.1064 - sparse_categorical_accuracy: 0.67

Train on 604 samples, validate on 152 samples
Epoch 1/100
 - 2s - loss: 0.5541 - sparse_categorical_accuracy: 0.4492 - val_loss: 0.8693 - val_sparse_categorical_accuracy: 0.4750
Epoch 2/100
 - 2s - loss: 0.1552 - sparse_categorical_accuracy: 0.4979 - val_loss: 1.1970 - val_sparse_categorical_accuracy: 0.4616
Epoch 3/100
 - 2s - loss: 0.1393 - sparse_categorical_accuracy: 0.4281 - val_loss: 1.0757 - val_sparse_categorical_accuracy: 0.4140
Epoch 4/100
 - 2s - loss: 0.1299 - sparse_categorical_accuracy: 0.4343 - val_loss: 0.9269 - val_sparse_categorical_accuracy: 0.4591
Epoch 5/100
 - 2s - loss: 0.1246 - sparse_categorical_accuracy: 0.4765 - val_loss: 0.9334 - val_sparse_categorical_accuracy: 0.4845
Epoch 6/100
 - 2s - loss: 0.1128 - sparse_categorical_accuracy: 0.4946 - val_loss: 0.8182 - val_sparse_categorical_accuracy: 0.5059
Epoch 7/100
 - 2s - loss: 0.1122 - sparse_categorical_accuracy: 0.5173 - val_loss: 0.8251 - val_sparse_categorical_accuracy: 0.5242
Epoch 8/100
 - 2s - loss: 0.10

Epoch 63/100
 - 2s - loss: 0.0201 - sparse_categorical_accuracy: 0.7419 - val_loss: 1.5982 - val_sparse_categorical_accuracy: 0.7427
Epoch 64/100
 - 2s - loss: 0.0128 - sparse_categorical_accuracy: 0.7441 - val_loss: 1.8731 - val_sparse_categorical_accuracy: 0.7450
Epoch 65/100
 - 2s - loss: 0.0114 - sparse_categorical_accuracy: 0.7464 - val_loss: 2.0317 - val_sparse_categorical_accuracy: 0.7474
Epoch 66/100
 - 2s - loss: 0.0232 - sparse_categorical_accuracy: 0.7486 - val_loss: 1.4878 - val_sparse_categorical_accuracy: 0.7495
Epoch 67/100
 - 2s - loss: 0.0231 - sparse_categorical_accuracy: 0.7505 - val_loss: 1.9614 - val_sparse_categorical_accuracy: 0.7512
Epoch 68/100
 - 2s - loss: 0.0226 - sparse_categorical_accuracy: 0.7521 - val_loss: 1.5420 - val_sparse_categorical_accuracy: 0.7527
Epoch 69/100
 - 2s - loss: 0.0241 - sparse_categorical_accuracy: 0.7535 - val_loss: 1.4190 - val_sparse_categorical_accuracy: 0.7541
Epoch 70/100
 - 2s - loss: 0.0152 - sparse_categorical_accuracy: 0.75

Epoch 7/100
 - 2s - loss: 0.2400 - sparse_categorical_accuracy: 0.4499 - val_loss: 0.9573 - val_sparse_categorical_accuracy: 0.4534
Epoch 8/100
 - 2s - loss: 0.2420 - sparse_categorical_accuracy: 0.4573 - val_loss: 0.9620 - val_sparse_categorical_accuracy: 0.4607
Epoch 9/100
 - 2s - loss: 0.2193 - sparse_categorical_accuracy: 0.4639 - val_loss: 0.9102 - val_sparse_categorical_accuracy: 0.4673
Epoch 10/100
 - 2s - loss: 0.2149 - sparse_categorical_accuracy: 0.4742 - val_loss: 0.9184 - val_sparse_categorical_accuracy: 0.4788
Epoch 11/100
 - 2s - loss: 0.1962 - sparse_categorical_accuracy: 0.4853 - val_loss: 0.9607 - val_sparse_categorical_accuracy: 0.4896
Epoch 12/100
 - 2s - loss: 0.1946 - sparse_categorical_accuracy: 0.4943 - val_loss: 0.8990 - val_sparse_categorical_accuracy: 0.4982
Epoch 13/100
 - 2s - loss: 0.1819 - sparse_categorical_accuracy: 0.5039 - val_loss: 1.0410 - val_sparse_categorical_accuracy: 0.5078
Epoch 14/100
 - 2s - loss: 0.1606 - sparse_categorical_accuracy: 0.5137 

...

Epoch 63/100
 - 1s - loss: 0.0510 - sparse_categorical_accuracy: 0.6803 - val_loss: 2.6165 - val_sparse_categorical_accuracy: 0.6815
Epoch 64/100
 - 1s - loss: 0.0483 - sparse_categorical_accuracy: 0.6826 - val_loss: 2.1442 - val_sparse_categorical_accuracy: 0.6835
Epoch 65/100
 - 1s - loss: 0.0411 - sparse_categorical_accuracy: 0.6849 - val_loss: 2.7970 - val_sparse_categorical_accuracy: 0.6860
Epoch 66/100
 - 1s - loss: 0.0346 - sparse_categorical_accuracy: 0.6875 - val_loss: 2.4697 - val_sparse_categorical_accuracy: 0.6888
Epoch 67/100
 - 1s - loss: 0.0355 - sparse_categorical_accuracy: 0.6901 - val_loss: 2.7254 - val_sparse_categorical_accuracy: 0.6912
Epoch 68/100
 - 1s - loss: 0.0409 - sparse_categorical_accuracy: 0.6924 - val_loss: 2.8143 - val_sparse_categorical_accuracy: 0.6934
Epoch 69/100
 - 1s - loss: 0.0360 - sparse_categorical_accuracy: 0.6947 - val_loss: 3.0940 - val_sparse_categorical_accuracy: 0.6957
Epoch 70/100
 - 1s - loss: 0.0313 - sparse_categorical_accuracy: 0.69

Epoch 7/100
 - 1s - loss: 0.3194 - sparse_categorical_accuracy: 0.3824 - val_loss: 1.0904 - val_sparse_categorical_accuracy: 0.3862
Epoch 8/100
 - 1s - loss: 0.3082 - sparse_categorical_accuracy: 0.3890 - val_loss: 1.0831 - val_sparse_categorical_accuracy: 0.3931
Epoch 9/100
 - 1s - loss: 0.3053 - sparse_categorical_accuracy: 0.3998 - val_loss: 1.0810 - val_sparse_categorical_accuracy: 0.4059
Epoch 10/100
 - 1s - loss: 0.2845 - sparse_categorical_accuracy: 0.4130 - val_loss: 1.1431 - val_sparse_categorical_accuracy: 0.4189
Epoch 11/100
 - 1s - loss: 0.2796 - sparse_categorical_accuracy: 0.4253 - val_loss: 1.0985 - val_sparse_categorical_accuracy: 0.4282
Epoch 12/100
 - 1s - loss: 0.2716 - sparse_categorical_accuracy: 0.4342 - val_loss: 1.1215 - val_sparse_categorical_accuracy: 0.4376
Epoch 13/100
 - 1s - loss: 0.2661 - sparse_categorical_accuracy: 0.4426 - val_loss: 1.1009 - val_sparse_categorical_accuracy: 0.4471
Epoch 14/100
 - 1s - loss: 0.2548 - sparse_categorical_accuracy: 0.4518 

Epoch 69/100
 - 1s - loss: 0.0419 - sparse_categorical_accuracy: 0.7086 - val_loss: 2.7755 - val_sparse_categorical_accuracy: 0.7095
Epoch 70/100
 - 1s - loss: 0.0527 - sparse_categorical_accuracy: 0.7106 - val_loss: 2.7795 - val_sparse_categorical_accuracy: 0.7114
Epoch 71/100
 - 1s - loss: 0.0649 - sparse_categorical_accuracy: 0.7124 - val_loss: 2.5485 - val_sparse_categorical_accuracy: 0.7130
Epoch 72/100
 - 1s - loss: 0.0547 - sparse_categorical_accuracy: 0.7140 - val_loss: 2.4470 - val_sparse_categorical_accuracy: 0.7147
Epoch 73/100
 - 1s - loss: 0.0392 - sparse_categorical_accuracy: 0.7156 - val_loss: 2.7190 - val_sparse_categorical_accuracy: 0.7165
Epoch 74/100
 - 1s - loss: 0.0538 - sparse_categorical_accuracy: 0.7174 - val_loss: 2.5274 - val_sparse_categorical_accuracy: 0.7180
Epoch 75/100
 - 2s - loss: 0.0354 - sparse_categorical_accuracy: 0.7191 - val_loss: 3.0138 - val_sparse_categorical_accuracy: 0.7197
Epoch 76/100
 - 1s - loss: 0.0395 - sparse_categorical_accuracy: 0.72

Epoch 13/100
 - 1s - loss: 0.0670 - sparse_categorical_accuracy: 0.4843 - val_loss: 0.8931 - val_sparse_categorical_accuracy: 0.4865
Epoch 14/100
 - 1s - loss: 0.0636 - sparse_categorical_accuracy: 0.4884 - val_loss: 0.9166 - val_sparse_categorical_accuracy: 0.4882
Epoch 15/100
 - 1s - loss: 0.0650 - sparse_categorical_accuracy: 0.4898 - val_loss: 0.9025 - val_sparse_categorical_accuracy: 0.4913
Epoch 16/100
 - 1s - loss: 0.0612 - sparse_categorical_accuracy: 0.4927 - val_loss: 0.9275 - val_sparse_categorical_accuracy: 0.4944
Epoch 17/100
 - 1s - loss: 0.0598 - sparse_categorical_accuracy: 0.4971 - val_loss: 0.9375 - val_sparse_categorical_accuracy: 0.4979
Epoch 18/100
 - 1s - loss: 0.0573 - sparse_categorical_accuracy: 0.4998 - val_loss: 0.9045 - val_sparse_categorical_accuracy: 0.5020
Epoch 19/100
 - 1s - loss: 0.0583 - sparse_categorical_accuracy: 0.5040 - val_loss: 0.9422 - val_sparse_categorical_accuracy: 0.5047
Epoch 20/100
 - 1s - loss: 0.0543 - sparse_categorical_accuracy: 0.50

Epoch 75/100
 - 1s - loss: 0.0142 - sparse_categorical_accuracy: 0.6763 - val_loss: 2.0079 - val_sparse_categorical_accuracy: 0.6773
Epoch 76/100
 - 1s - loss: 0.0113 - sparse_categorical_accuracy: 0.6787 - val_loss: 2.1511 - val_sparse_categorical_accuracy: 0.6799
Epoch 77/100
 - 1s - loss: 0.0118 - sparse_categorical_accuracy: 0.6811 - val_loss: 2.3148 - val_sparse_categorical_accuracy: 0.6821
Epoch 78/100
 - 1s - loss: 0.0089 - sparse_categorical_accuracy: 0.6834 - val_loss: 2.5884 - val_sparse_categorical_accuracy: 0.6845
Epoch 79/100
 - 1s - loss: 0.0139 - sparse_categorical_accuracy: 0.6857 - val_loss: 2.3911 - val_sparse_categorical_accuracy: 0.6866
Epoch 80/100
 - 1s - loss: 0.0118 - sparse_categorical_accuracy: 0.6880 - val_loss: 2.4555 - val_sparse_categorical_accuracy: 0.6890
Epoch 81/100
 - 1s - loss: 0.0132 - sparse_categorical_accuracy: 0.6901 - val_loss: 2.3562 - val_sparse_categorical_accuracy: 0.6909
Epoch 82/100
 - 1s - loss: 0.0124 - sparse_categorical_accuracy: 0.69